# March Code Pudding

### March Maddness Bracket

In [152]:
import pandas as pd

### Data Preprocessing:

In [153]:
Defense = pd.read_csv('INT _ KenPom _ Defense.csv')
Efficiency = pd.read_csv('INT _ KenPom _ Efficiency.csv')
Height = pd.read_csv('INT _ KenPom _ Height.csv')
Misc = pd.read_csv('INT _ KenPom _ Miscellaneous Team Stats.csv')
Offense = pd.read_csv('INT _ KenPom _ Offense.csv')
Points = pd.read_csv('INT _ KenPom _ Point Distribution.csv')
Summary = pd.read_csv('INT _ KenPom _ Summary.csv')

In [154]:
# changing Team to TeamName in Efficiency df so easier merge
Efficiency.rename(columns={"Team": "TeamName"}, inplace=True)

In [155]:
# One data frame from everything but Coach
dfs = [Defense, Efficiency, Height, Misc, Offense, Points, Summary]

# Merge all DataFrames on 'Season' and 'TeamName' using outer join to keep all data
Data = dfs[0]
for df in dfs[1:]:
    Data = Data.merge(df, on=["Season", "TeamName"], how="outer")

# Display the first few rows
display(Data.head())

,Season,TeamName,eFGPct_x,RankeFGPct_x,TOPct_x,RankTOPct_x,ORPct_x,RankORPct_x,FTRate_x,RankFTRate_x,...,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM
0,2021,Abilene Christian,46.012510,22.0,26.492119,1.0,27.851140,168.0,41.125880,328.0,...,104.1510,106.0,101.6800,184.0,86.9821,1.0,91.5665,26.0,10.11380,84.0
1,2021,Air Force,57.372040,343.0,21.779096,41.0,35.537190,343.0,32.849503,206.0,...,92.1076,319.0,94.3993,302.0,112.5540,337.0,110.9120,326.0,-16.51310,332.0
2,2021,Akron,48.622800,104.0,16.434850,300.0,28.836634,208.0,27.314461,68.0,...,108.5670,41.0,108.6010,73.0,102.9900,221.0,101.4570,159.0,7.14365,112.0
3,2021,Alabama,45.762299,16.0,20.502639,77.0,29.160305,222.0,30.686800,158.0,...,106.7570,63.0,112.8680,30.0,93.7315,20.0,87.7822,3.0,25.08570,9.0
4,2021,Alabama A&M,43.995511,3.0,17.983632,220.0,29.371817,232.0,37.373737,297.0,...,88.2408,341.0,84.7659,354.0,94.3829,26.0,105.8030,250.0,-21.03720,348.0


In [156]:
# The ranked data won't correlate as closely from year to year
Data.drop(columns=[col for col in Data.columns if 'rank' in col.lower()], inplace=True)

In [157]:
# selecting only this years data
Data = Data[Data['Season'] == 2025]

<div class="alert alert-info">
Could add more years? maybe up to 2022 when NIL was began
</div>

In [158]:
#cleaning cloumn names
Data.columns = Data.columns.str.lower().str.replace(' ', '_')

In [159]:
# breif look at all the column names and count
columns = Data.columns
display(columns)
len(columns)

Index(['season', 'teamname', 'efgpct_x', 'topct_x', 'orpct_x', 'ftrate_x',
       'conference', 'adjusted_temo', 'raw_tempo',
       'adjusted_offensive_efficiency', 'raw_offensive_efficiency',
       'adjusted_defensive_efficiency', 'raw_defensive_efficiency',
       'avg_possession_length_(offense)', 'avg_possession_length_(defense)',
       'avgheight', 'centerheight', 'pfheight', 'sfheight', 'sgheight',
       'pgheight', 'effectiveheight', 'experience', 'bench', 'centerpts',
       'pfpts', 'sfpts', 'sgpts', 'pgpts', 'centeror', 'pfor', 'sfor', 'sgor',
       'pgor', 'centerdr', 'pfdr', 'sfdr', 'sgdr', 'pgdr', 'fg2pct', 'fg3pct',
       'ftpct', 'blockpct', 'oppfg2pct', 'oppfg3pct', 'oppftpct',
       'oppblockpct', 'fg3rate', 'oppfg3rate', 'arate', 'opparate', 'stlrate',
       'oppstlrate', 'dfp', 'nstrate', 'oppnstrate', 'efgpct_y', 'topct_y',
       'orpct_y', 'ftrate_y', 'offft', 'off2ptfg', 'off3ptfg', 'defft',
       'def2ptfg', 'def3ptfg', 'tempo', 'adjtempo', 'oe', 'adjoe

73

In [160]:
# reducing columns
columns_keep = [
#    'season',  # commented it out since only looking at 2025
    'teamname',  
    'efgpct_x',  
    'topct_x',  
    'orpct_x',  
    'ftrate_x',  
    'adjusted_temo',  
    'adjusted_offensive_efficiency',  
    'adjusted_defensive_efficiency',  
    'experience',  
    'bench',  
    'fg2pct',  
    'fg3pct',  
    'ftpct',  
]

# reducing data to these features
Data = Data[columns_keep]


### Explanation of Columns in the Data

The following columns have been selected to analyze team performance for the 2025 season:

1. **teamname**: The name of the team participating in the analysis.
2. **efgpct_x**: The effective field goal percentage, a metric that adjusts the field goal percentage to account for the added value of three-point shots.
3. **topct_x**: The turnover percentage, which represents the percentage of offensive possessions that end in a turnover.
4. **orpct_x**: The offensive rebound percentage, indicating the percentage of available offensive rebounds a team captures.
5. **ftrate_x**: The free throw rate, which measures the number of free throws taken relative to the number of field goal attempts.
6. **adjusted_temo**: The adjusted tempo, which reflects the team's pace of play, considering the tempo of both the team and its opponents.
7. **adjusted_offensive_efficiency**: The adjusted offensive efficiency, which measures the team's offensive output per 100 possessions, accounting for the strength of the team's opponents.
8. **adjusted_defensive_efficiency**: The adjusted defensive efficiency, which measures how effectively the team defends per 100 possessions, considering the strength of their opponents.
9. **experience**: The average years of experience for the players on the team, representing their familiarity and maturity in the game.
10. **bench**: The total contribution from the bench players, reflecting the depth of the team's roster.
11. **fg2pct**: The field goal percentage for two-point attempts, indicating how efficiently the team scores inside the arc.
12. **fg3pct**: The field goal percentage for three-point attempts, showing the team's shooting efficiency from beyond the arc.
13. **ftpct**: The free throw percentage, measuring how successful the team is in converting free throws.



# trialing setting up data from modeling

In [ ]:
# 64 teams list
teams_64 = ['Team1', 'Team2', 'Team3', 'Team4'
            
            # Define matchups for each team (as an example)
matchups_dict = {
    'Team1': ['Matchup_1', 'Matchup_2', 'Matchup_3'],
    'Team2': ['Matchup_4', 'Matchup_5', 'Matchup_6']}

# Function to create matchup data for all teams
def generate_all_matchups(teams_list, matchups_dict, data):
    all_matchups = []

    # Loop through each team and generate matchups
    for team in teams_list:
        matchups = matchups_dict.get(team, [])
        
        for opponent in matchups:
            # Filter the data for the current team and opponent
            team_data = data[data['teamname'] == team].drop(columns=['teamname'])
            opponent_data = data[data['teamname'] == opponent].drop(columns=['teamname'])
            
            # Rename columns for both teams to distinguish them
            team_data.columns = [col + f'_{team}' for col in team_data.columns]
            opponent_data.columns = [col + f'_{opponent}' for col in opponent_data.columns]
            
            # Concatenate the two teams' data side by side
            matchup = pd.concat([team_data.reset_index(drop=True), opponent_data.reset_index(drop=True)], axis=1)
            
            # Add a column to show the winner of the matchup (this can be determined based on specific criteria)
            # Example: If Team1's adjusted_offensive_efficiency > Team2's, then Team1 wins
            winner = team if team_data['adjusted_offensive_efficiency'].values[0] > opponent_data['adjusted_offensive_efficiency'].values[0] else opponent
            matchup['winner'] = winner
            
            # Add matchup data to the list
            all_matchups.append(matchup)
    
    # Combine all matchups into a single DataFrame
    final_matchups_df = pd.concat(all_matchups, ignore_index=True)
    
    return final_matchups_df

# Example: Generate matchups for all teams
final_matchups = generate_all_matchups(teams_64, matchups_dict, Data)

# Display the final matchups DataFrame
display(final_matchups)

In [161]:
Duke = Data[Data['teamname'] == 'Duke']
display(Duke)

Wake_Forest = Duke = Data[Data['teamname'] == 'Wake Forest']
display(Wake_Forest)

#duke won this so T1_win = 1 ( = 0 if lost?)

,teamname,efgpct_x,topct_x,orpct_x,ftrate_x,adjusted_temo,adjusted_offensive_efficiency,adjusted_defensive_efficiency,experience,bench,fg2pct,fg3pct,ftpct
1114,Duke,44.193742,17.962362,26.115242,25.631769,66.1,128.9,89.8,1.92886,31.37,57.49203,38.580645,78.158845


,teamname,efgpct_x,topct_x,orpct_x,ftrate_x,adjusted_temo,adjusted_offensive_efficiency,adjusted_defensive_efficiency,experience,bench,fg2pct,fg3pct,ftpct
1385,Wake Forest,47.459165,19.785613,31.200787,31.69994,67.3,109.2,98.6,2.14154,21.47,54.423077,28.62254,75.59322


In [162]:
# Remove 'teamname' column to avoid duplication when concatenating
Duke = Duke.drop(columns=['teamname'])
Wake_Forest = Wake_Forest.drop(columns=['teamname'])

# Rename columns to include team names (for clarity)
Duke.columns = [col + '_Duke' for col in Duke.columns]
Wake_Forest.columns = [col + '_Wake_Forest' for col in Wake_Forest.columns]

# Append the two rows together side by side
match_up = pd.concat([Duke.reset_index(drop=True), Wake_Forest.reset_index(drop=True)], axis=1)

# Display the result
display(match_up)

,efgpct_x_Duke,topct_x_Duke,orpct_x_Duke,ftrate_x_Duke,adjusted_temo_Duke,adjusted_offensive_efficiency_Duke,adjusted_defensive_efficiency_Duke,experience_Duke,bench_Duke,fg2pct_Duke,...,orpct_x_Wake_Forest,ftrate_x_Wake_Forest,adjusted_temo_Wake_Forest,adjusted_offensive_efficiency_Wake_Forest,adjusted_defensive_efficiency_Wake_Forest,experience_Wake_Forest,bench_Wake_Forest,fg2pct_Wake_Forest,fg3pct_Wake_Forest,ftpct_Wake_Forest
0,47.459165,19.785613,31.200787,31.69994,67.3,109.2,98.6,2.14154,21.47,54.423077,...,31.200787,31.69994,67.3,109.2,98.6,2.14154,21.47,54.423077,28.62254,75.59322
